In [1]:
import os
import random
import plotly 
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import koreanize_matplotlib
import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor


warnings.filterwarnings("ignore")
# pd.options.display.max_rows = None
# pd.options.display.max_columns = None
%config InlineBackend.figure_format = 'retina'


# plt.style.use('dark_background')
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'NanumGothic'

In [20]:
train = pd.read_csv('train_모든컬럼다있는버전3.csv')
test = pd.read_csv('test_모든컬럼다있는버전3.csv')
submission = pd.read_csv('sample_submission.csv')

In [21]:
train = train.drop(columns=['키(cm)', '체중(kg)', '심박수(BPM)', '체온(C)', 'BMI', 'BMI_scaled'])
test = test.drop(columns=['키(cm)', '체중(kg)', '심박수(BPM)', '체온(C)', 'BMI', 'BMI_scaled'])

In [22]:
# 종속 변수와 독립 변수 분리
X = train[['운동시간(분)', '나이', '키_scaled', '체중(kg)_scaled', '심박수(BPM)_scaled', '체온(C)_scaled']]

# VIF 계산

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['features'] = X.columns
print(vif)


   VIF Factor         features
0    8.020428          운동시간(분)
1    6.503117               나이
2   12.836019         키_scaled
3   12.939352    체중(kg)_scaled
4    2.734227  심박수(BPM)_scaled
5    3.110449     체온(C)_scaled


In [25]:
# 종속 변수와 독립 변수 분리
X = test[['운동시간(분)', '나이', '체중(kg)_scaled', '심박수(BPM)_scaled', '체온(C)_scaled', '키_scaled']]

# VIF 계산

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['features'] = X.columns
print(vif)

   VIF Factor         features
0    7.919400          운동시간(분)
1    6.464654               나이
2   12.576436    체중(kg)_scaled
3    2.740863  심박수(BPM)_scaled
4    3.097452     체온(C)_scaled
5   12.483087         키_scaled


In [15]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

In [16]:
X = train.drop(columns=['소모칼로리'])
y = train['소모칼로리']

In [ ]:
model = CatBoostRegressor(loss_function='RMSE', depth=6, iterations=1000, l2_leaf_reg=1, learning_rate=0.1) 
model.fit(X, y)

# 모델 예측
X_test = test 
y_predict = model.predict(X_test)
y_predict[:5]

In [18]:
submission = pd.read_csv('sample_submission.csv')
submission['Calories_Burned'] = y_predict

In [19]:
submission.to_csv('캣부스트4.csv', index = False)